In [1]:
!pip install django python-geohash django-filter dj_database_url psycopg2  model_mommy  django-leaflet django-rest-framework djangorestframework-gis
!pip install https://github.com/sigma-geosistemas/mommy_spatial_generators/archive/master.zip
!pip install -U ..

Processing /notebooks
Requirement already up-to-date: django>=1.8 in /usr/local/lib/python2.7/dist-packages (from django-geohash-cluster==0.1.0)
Requirement already up-to-date: pytz in /usr/local/lib/python2.7/dist-packages (from django>=1.8->django-geohash-cluster==0.1.0)
  Found existing installation: django-geohash-cluster 0.1.0
    Uninstalling django-geohash-cluster-0.1.0:
      Successfully uninstalled django-geohash-cluster-0.1.0
  Running setup.py install for django-geohash-cluster ... done


In [2]:
import os
os.environ['DATABASE_URL'] = 'postgis://geohash:test@db:5432/geohash'
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "example.settings")

'example.settings'

In [3]:
import django
django.setup()
!python manage.py migrate


Operations to perform:
  Apply all migrations: admin, auth, contenttypes, example, sessions
Running migrations:
  No migrations to apply.


In [4]:
from example import models

In [28]:
from model_mommy import mommy
places = mommy.make(models.Place, _quantity=10000)

In [6]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [42]:
from ipywidgets import interact
import ipywidgets as widgets

from example.views import PointSerializer

zoomToGeoHashPrecision = {
    0: 1,
    1: 2,
    2: 2,
    3: 2,
    4: 3,
    5: 3,
    6: 4,
    7: 4,
    8: 4,
    9: 5,
    10: 5,
    11: 6,
    12: 6,
    13: 6,
    14: 7,
    15: 7,
    16: 7,
    17: 7,
    18: 7,
    19: 7,
    20: 7,
    21: 7
}

def cluster(zoom):
    precision = zoomToGeoHashPrecision[zoom]
    
    places = models.Place.objects.cluster_points(precision)
    serializer = PointSerializer(places, many=True)
    return serializer.data


def map_widgets(lat=0.0, lon=0.0, zoom=0):
    M.layers.annotation.clear_annotations()
    M.set_center(lon, lat, zoom)

    for i in cluster(zoom)['features']:
        M.add_annotation('point', i['geometry']['coordinates'])
  
interact(map_widgets, lat=(-90.0, 90.0), lon=(-180.0, 180.0), zoom=(0, 9))

interactive(children=(FloatSlider(value=0.0, description=u'lat', max=90.0, min=-90.0), FloatSlider(value=0.0, description=u'lon', max=180.0, min=-180.0), IntSlider(value=0, description=u'zoom', max=9), Output()), _dom_classes=('widget-interact',))

<function __main__.map_widgets>

In [43]:
import timeit

In [ ]:
timeit.timeit('cluster(0)', setup='from __main__ import cluster', number=1000)

16.0557119846344

In [ ]:
timeit.timeit('cluster(1)', setup='from __main__ import cluster', number=1000)

In [ ]:
timeit.timeit('cluster(0)', setup='from __main__ import cluster', number=1000)